Directories size

In [2]:
import os, shutil

train_dir = './dataset/train'
validation_dir = './dataset/validation'
test_dir = './dataset/test'
train_glob_red = './dataset/train/trainglob/red'
train_glob_blue = './dataset/train/trainglob/blue'
val_glob_red = './dataset/validation/valglob/red'
val_glob_blue = './dataset/validation/valglob/blue'
test_glob_blue = './dataset/test/testglob/blue'
test_glob_red = './dataset/test/testglob/red'
print('total training red images:', len(os.listdir(train_glob_red)))
print('total training blue images:', len(os.listdir(train_glob_blue)))
print('total validation red images:', len(os.listdir(val_glob_red)))
print('total validation blue images:', len(os.listdir(val_glob_blue)))
print('total testing red images:', len(os.listdir(test_glob_red)))
print('total testing blue images:', len(os.listdir(test_glob_blue)))

total training red images: 0
total training blue images: 0
total validation red images: 0
total validation blue images: 0
total testing red images: 0
total testing blue images: 0


Data preprocessing

In [3]:
from keras.utils import image_dataset_from_directory

IMG_SIZE = 150

train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32)

validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32)

2025-05-19 13:42:02.557773: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 13:42:02.575084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747658522.586534   25783 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747658522.589781   25783 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747658522.600228   25783 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Found 0 files belonging to 2 classes.


ValueError: No images found in directory ./dataset/train. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

Shape of each batch

In [ ]:
for data_batch, labels_batch in train_dataset:
 print('data batch shape:', data_batch.shape)
 print('labels batch shape:', labels_batch.shape)
 break

Viewing first 5 images of the first batch

In [ ]:
import matplotlib.pyplot as plt

for data_batch, _ in train_dataset.take(1):
 for i in range(5):
    plt.imshow(data_batch[i].numpy().astype("uint8"))
    plt.show()

Creating NN

In [ ]:
from tensorflow import keras
from keras import layers
from keras import models

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Compiling NN

In [ ]:
import tensorflow as tf
model.compile(
 loss='binary_crossentropy',
 optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
 metrics=['acc'])

Training the model

In [ ]:
history = model.fit(
 train_dataset,
 epochs=30,
 validation_data=validation_dataset)

In [ ]:
Loading and Testing the model

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/models/04_CNN_02_CatsAndDogs_01_cnn_from_scratch.h5')

Loss and Accuracy

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
